This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [4]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [5]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "test"    # e.g. './muffin-vs-chihuahua/test'

In [6]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [8]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [10]:
# IMPROVED CNN MODEL ARCHITECTURE WITH REGULARIZATION AND DROPOUT

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting and L2 regularization
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', 
                  kernel_regularizer=tf.keras.regularizers.l2(0.001),
                  input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu',
                  kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu',
                 kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [11]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.6444 - loss: 0.8682 - val_accuracy: 0.7471 - val_loss: 0.7178
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 57s 484ms/step - accuracy: 0.7529 - loss: 0.6293 - val_accuracy: 0.8011 - val_loss: 0.5330
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 57s 475ms/step - accuracy: 0.7928 - loss: 0.5472 - val_accuracy: 0.8392 - val_loss: 0.5003
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 56s 473ms/step - accuracy: 0.8152 - loss: 0.5040 - val_accuracy: 0.8815 - val_loss: 0.4177
Epoch 5/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 56s 470ms/step - accuracy: 0.8168 - loss: 0.4887 - val_accuracy: 0.8561 - val_loss: 0.4449
Epoch 6/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 56s 467ms/step - accuracy: 0.8345 - loss: 0.4536 - val_accuracy: 0.8169 - val_loss: 0.4517
Epoch 7/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 466ms/step - accuracy: 0.8424 - loss: 0.4282 - val_accuracy: 0.8783 - val_loss: 0.3640
Epoch 8/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 466ms/step - accuracy: 0.8471 - loss: 0.4

In [13]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 27s 750ms/step - accuracy: 0.8699 - loss: 0.3615
Test Accuracy: 0.8699324131011963


In [14]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # pred is P(Muffin) based on flow_from_directory's alphabetical order
    prob_muffin = model.predict(img_array)[0,0]
    
    # Determine the predicted class and its confidence
    if prob_muffin >= 0.5:
        label = "Muffin"
        confidence = prob_muffin
    else:
        label = "Chihuahua"
        # P(Chihuahua) = 1 - P(Muffin)
        confidence = 1.0 - prob_muffin
        
    print(f"Prediction: {label} (confidence: {confidence:.2f})")

In [16]:
# Example usage:
predict_image("test/chihuahua/img_0_22.jpg")
predict_image("test/muffin/img_0_32.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


Prediction: Chihuahua (confidence: 0.85)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Prediction: Muffin (confidence: 0.98)
